In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import learning_curve
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
# from tpot import TPOTRegressor
from sklearn.ensemble import GradientBoostingRegressor
import warnings

In [7]:
df_air = pd.read_csv('../../raw_data/Gida Will --_ Water & Air Data  - Air Data.csv')

In [8]:
df_air.head()

,Date,Température 1 (◦C),Température 2 (◦C),Température 3 (◦C),Humidité 1 (%),Humidité 2 (%),Humidité 3 (%),CO2 (ppm),Unnamed: 8
0,8/8/21 0:00,$24.10,23.3,24.7,80.0,80.0,79.0,-,NaN
1,8/8/21 0:30,24,23.2,24.6,80.0,80.0,79.0,-,NaN
2,8/8/21 1:00,23.9,23.1,24.5,80.0,80.0,78.0,-,NaN
3,8/8/21 1:30,23.9,23.1,24.5,80.0,80.0,78.0,-,NaN
4,8/8/21 2:00,23.8,23.0,24.5,79.0,79.0,78.0,-,NaN


In [9]:
df_air.columns

Index(['Date', 'Température 1 (◦C)', 'Température 2 (◦C)',
       'Température 3 (◦C)', 'Humidité 1 (%)', 'Humidité 2 (%)',
       'Humidité 3 (%)', 'CO2 (ppm)', 'Unnamed: 8'],
      dtype='object')

In [10]:
df_g1 = pd.read_csv('../../raw_data/Gida Will --_ Water & Air Data  - Groupe 1 (Twr 1,2,3,4).csv', low_memory=False)
df_g1.columns = df_g1.iloc[0] 
df_g1 = df_g1[1:]

In [11]:
df_g1.head()

,Date,nutriments (S21),Avg. E.C,Water Temp (S22),Avg. TEMP (°F),pH (S23),Avg. pH,"twr level (in "" S31)","Avg. Tower Level ("")",Dissolved O2,...,pH- Pump (A43),pH+ Pump (A44),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,mS/cm,NaN,°F,NaN,NaN,NaN,NaN,NaN,NaN,...,%,%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,25-Feb-2021 0:30:00,1.60,1.59,65.90,66.71,6.00,6.06,15.18,15.17,NaN,...,1,0,8.2,0,Off,NaN,NaN,NaN,NaN,NaN
3,25-Feb-2021 1:00:00,1.61,NaN,65.70,NaN,6.00,NaN,15.16,NaN,NaN,...,0,0.1,0.0,0,Off,NaN,NaN,NaN,NaN,NaN
4,25-Feb-2021 1:30:00,1.60,NaN,65.10,NaN,6.00,NaN,15.13,NaN,NaN,...,0.2,0.1,1.3,0,Off,NaN,NaN,NaN,NaN,NaN
5,25-Feb-2021 2:00:00,1.60,NaN,64.90,NaN,5.99,NaN,15.12,NaN,NaN,...,0,0,0.0,0,Off,NaN,NaN,NaN,NaN,NaN


In [12]:
print(df_g1.columns[-8:])
print(f'\n length: {len(df_g1.columns)}')

Index([nan, nan, nan, nan, nan, nan, nan, nan], dtype='object', name=0)

 length: 32


In [13]:
df_g2 = pd.read_csv('../../raw_data/Gida Will --_ Water & Air Data  - Groupe 2 (Twr 5,6,7,8).csv', low_memory=False)
df_g2.columns = df_g2.iloc[0] 
df_g2 = df_g2[1:]

In [14]:
print(df_g2.columns)
print(f'\n length: {len(df_g2.columns)}')

Index([                  'Date',              'PSC (S11)',
                     'Avg. PSC',       'nutriments (S21)',
                     'Avg. E.C',       'Water Temp (S24)',
               'Avg. TEMP (°F)',               'pH (S23)',
                       'Avg pH', 'twr  level (in " S31) ',
         'Avg. Tower Level (")',        'Flowswitch (D1)',
              'tower valv (R1)',        'tower pump (R2)',
              'Lvl ntrlck (R3)',        'PRGHGHCOND (R4)',
                   'Alarm (R5)',            'Manual (R6)',
                  'Manual (R7)',            'Manual (R8)',
               'PSC pump (A41)',           'nutA&B (A42)',
               'pH- Pump (A43)',         'pH+ Pump (A44)',
                   'Off / On 1',             'Off / On 2',
                   'Off / On 3',               'Comments',
                            nan,                      nan,
                            nan],
      dtype='object', name=0)

 length: 31


In [28]:
df_g2['Lvl ntrlck (R3)'].value_counts()

Off    48891
On      2405
Name: Lvl ntrlck (R3), dtype: int64

In [21]:
df_g56 = pd.read_csv('../../raw_data/Gida Will --_ Water & Air Data  - Group 5 & 6 (Twr 17 to 24).csv', low_memory=False)
df_g56.columns = df_g56.iloc[0] 
df_g56 = df_g56[1:]

In [22]:
print(df_g56.columns)
print(f'\n length: {len(df_g56.columns)}')

Index([            'Date',   'G6 nutAB (S11)',    'G6 temp (S12)',
            'G6 Ph (S13)',    'G6 Temp (S14)',   'G5 nutAB (S21)',
          'G5 temp (S22)',      'G5 Ph (S23)',    'G5 Temp (S24)',
            'G6 dO (S31)',  'G6 TnkLVL (S32)',  'G5 TnkLVL (S33)',
            'G5 dO (S34)',     'G6 Flow (D1)',     'G5 Flow (D2)',
       'G6 tnkFEED (R11)',          '- (R12)', 'G5 TnkFEED (R13)',
                '- (R14)',          '- (R21)',          '- (R22)',
                '- (R23)',          '- (R24)', 'G6 Tnk PMP (R31)',
                '- (R32)', 'G5 Tnk PMP (R33)',          '- (R34)',
       'G6 nAB pmp (A41)', 'G6 Ph pump (A42)', 'G5 nAB pmp (A43)',
       'G5 Ph pump (A44)',                nan],
      dtype='object', name=0)

 length: 32


In [40]:
import re

def key_cleaner(df):
    cleaned_keys = []
    keys = df.columns
    for i in keys:
        print(i[0])
        cleaned_keys.append(i.strip().lower().replace(' ','_'))
    return cleaned_keys

In [42]:
!pwd

/home/williamjwall/code/gidahusa/Aquaverti-Data-Analysis/notebooks


In [58]:
import pandas as pd
import glob
import os

path = r'../../raw_data'
all_files = glob.glob(os.path.join(path , "*.csv"))

lista = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    lista.append(df)

frame = pd.concat(lista, axis=0, ignore_index=True)

/tmp/ipykernel_4076/3421594069.py:11: DtypeWarning: Columns (1,2,3,4,5,6,7,8,22,23,25,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename, index_col=None, header=0)
/tmp/ipykernel_4076/3421594069.py:11: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,19,20,21,22,23,27,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename, index_col=None, header=0)
/tmp/ipykernel_4076/3421594069.py:11: DtypeWarning: Columns (1,2,4,6,10,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename, index_col=None, header=0)


In [63]:
frame

,Unique Id 1,Active or Inactive,Product:,Todays date:,Transplant date:,Seeds planted date:,Total days in Nursery:,Total days in GrowOff: (auto-updated),Total cycle:,Transplanted by:,...,Total # of Oasis openings planted into:,Total # of Oasis:,# of plants in batch: (auto-updated),Planted by:,Days in nursery: (auto-updated),Click to enter tossed plants from batch details:,% of plants tossed from the batch before GrowOff (auto-updated),# of plants tossed:,F&D Location:,Operations Note:
0,436fa514,Inactive,Arugula (Standard),12/7/2022,11/16/2020,10/27/2020,20.0,751.0,771,"Gerald Sara , Sarah Flack",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,071275be,Inactive,Kale (Starbor),12/7/2022,11/16/2020,10/27/2020,20.0,751.0,771,"Gerald Sara , Sarah Flack",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,042707fc,Inactive,Kale (Starbor),12/7/2022,11/16/2020,10/27/2020,20.0,751.0,771,"Gerald Sara , Sarah Flack",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,72020ff2,Inactive,Lettuce (Muir),12/7/2022,11/16/2020,10/28/2020,19.0,751.0,770,"Gerald Sara , Sarah Flack",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2519ee37,Inactive,Lettuce (Rouxai),12/7/2022,11/16/2020,10/28/2020,19.0,751.0,770,"Gerald Sara , Sarah Flack",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246497,e1e9f951,Active,Purslane,2022-12-07,NaN,2022-11-01,NaN,NaN,NaN,NaN,...,1104.0,4.0,#REF!,Raffi Khaloyan,36.0,NaN,0%,NaN,NaN,NaN
246498,c6e88737,Active,Lettuce (Butterhead),2022-12-07,NaN,2022-12-05,NaN,NaN,NaN,NaN,...,1380.0,5.0,#REF!,Raffi Khaloyan,2.0,NaN,0%,NaN,Tower 24 (level 2),NaN
246499,b656c311,Active,Kale (Starbor),2022-12-07,NaN,2022-12-05,NaN,NaN,NaN,NaN,...,2484.0,9.0,#REF!,Raffi Khaloyan,2.0,NaN,0%,NaN,Tower 24 (level 2),NaN
246500,79c3ade2,Active,Lettuce (Muir),2022-12-07,NaN,2022-12-05,NaN,NaN,NaN,NaN,...,17112.0,62.0,#REF!,Raffi Khaloyan,2.0,NaN,0%,NaN,"Tower 8 (level 1) , Tower 18 (Level 1) , Tower...",NaN


In [25]:
df_g56.dropna(axis=0)

,Date,G6 nutAB (S11),G6 temp (S12),G6 Ph (S13),G6 Temp (S14),G5 nutAB (S21),G5 temp (S22),G5 Ph (S23),G5 Temp (S24),G6 dO (S31),...,- (R24),G6 Tnk PMP (R31),- (R32),G5 Tnk PMP (R33),- (R34),G6 nAB pmp (A41),G6 Ph pump (A42),G5 nAB pmp (A43),G5 Ph pump (A44),NaN
64204,31-Aug-2022 10:36:00,0.64,73.1,7.86,72.5,1.79,70.1,5.96,70,5.68,...,0,0,On,Off,On,Off,0,0,43.5,5.3
64205,31-Aug-2022 10:38:00,0.64,73.1,7.97,70.3,1.79,70.2,5.96,70,5.69,...,0,0,On,Off,On,Off,0,0,34.1,5.2
64206,31-Aug-2022 10:40:00,0.64,73.1,8.16,72,1.79,70.3,5.95,70.1,5.74,...,0,0,On,Off,On,Off,0,0,42.8,5.1
64207,31-Aug-2022 10:42:00,0.64,73.1,8.53,73.4,1.79,70.4,5.95,70.1,5.69,...,0,0,On,Off,On,Off,0,0,29,5.1
64208,31-Aug-2022 10:44:00,0.64,73.1,6.01,72.7,1.79,70.4,5.95,70.2,5.68,...,0,0,On,Off,On,Off,0,0,25.9,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131550,04-Dec-2022 23:56:00,1.8,20.90,5.85,20.60,1.31,21.10,5.87,20.90,5.65,...,0,0,On,Off,On,Off,0,1.6,100,2.4
131551,04-Dec-2022 23:58:00,1.8,20.90,5.85,20.60,1.3,21.10,5.87,20.90,5.5,...,0,0,On,Off,On,Off,0,1.6,100,2.3
131552,05-Dec-2022 0:00:00,1.8,20.90,5.85,20.60,1.28,21.10,5.89,20.70,5.63,...,0,0,On,Off,On,Off,6.7,1.6,100,3.1
131553,05-Dec-2022 0:02:00,1.8,20.90,5.85,20.60,1.27,21.00,5.89,20.70,5.61,...,0,0,On,Off,On,Off,6.5,1.6,100,3.0


In [26]:
df_g56.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131554 entries, 1 to 131554
Data columns (total 32 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Date              131553 non-null  object 
 1   G6 nutAB (S11)    131554 non-null  object 
 2   G6 temp (S12)     131554 non-null  object 
 3   G6 Ph (S13)       131553 non-null  object 
 4   G6 Temp (S14)     131554 non-null  object 
 5   G5 nutAB (S21)    131554 non-null  object 
 6   G5 temp (S22)     131554 non-null  object 
 7   G5 Ph (S23)       131553 non-null  object 
 8   G5 Temp (S24)     131554 non-null  object 
 9   G6 dO (S31)       131217 non-null  object 
 10  G6 TnkLVL (S32)   131194 non-null  object 
 11  G5 TnkLVL (S33)   131553 non-null  object 
 12  G5 dO (S34)       131553 non-null  object 
 13  G6 Flow (D1)      131553 non-null  object 
 14  G5 Flow (D2)      131553 non-null  object 
 15  G6 tnkFEED (R11)  131238 non-null  object 
 16  - (R12)           13